<a href="https://colab.research.google.com/github/liviator/Programmation_blockchain_td2/blob/main/TD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import secrets
import hashlib
import hmac
import subprocess

In [ ]:
def check_entropy():
    words = []
    print("Please type the words of your seed")
    for i in range(12):
        word = input()
        words.append(word)
        if i == 0 and len(word.split()) == 12:
          words = word.split()
          break
    entropy = check_format(words)
    if entropy == -1:
      print("The seed is not valid")
    else:
      print("The entropy is " + entropy)
    print("")
    return entropy

In [ ]:
def check_format(words):
    response = -1
    splitted = get_bag(words)
    united = ''.join(splitted)
    check_sum = united[len(united)-4:]
    entropy = united[:len(united)-4]
    if(len(entropy) == 128):       
        encoded_entropy_bits = int(entropy,2).to_bytes(128 // 8, byteorder='big')
        hashed_entropy = hashlib.sha256(encoded_entropy_bits)
        output = hashed_entropy.hexdigest()
        output = format(int(output,16),'0256b')
        if(output[:4] == check_sum):
            response = entropy
    return response
 

In [ ]:
def get_bag(words):
    bag = []
    list_of_words = get_words()
    index = 0
    for elem in words:
        if( elem in list_of_words):
            index = list_of_words.index(elem)
            bag.append(format(index,'011b'))
    return bag

In [121]:
def gen_entropy(size = 128):
  entropy = secrets.randbits(size)
  print("Random number :")
  print(entropy)
  print(" ")
  return add_padding(entropy)

In [ ]:
def add_padding(entropy):
  entropy_bits = format(entropy,'0128b')
  print("Random number in binary :")
  print(entropy_bits)
  print(" ")
  return entropy_bits

In [ ]:
def gen_checksum(entropy_bits):
  encoded_entropy_bits = int(entropy_bits,2).to_bytes(128 // 8, byteorder='big')
  hashed_entropy = hashlib.sha256(encoded_entropy_bits)
  output = hashed_entropy.hexdigest()
  output = format(int(output,16),'0256b')
  print("Output of the hash in binary :")
  print(output)
  print(" ")
  return output

In [ ]:
def add_checksum_to_entropy(entropy):
  entropy_bits_finale = entropy + gen_checksum(entropy)[:4]
  print("Entropy + Checksum :")
  print(entropy_bits_finale + "\n")
  return entropy_bits_finale

In [ ]:
def get_words():
  return open('/content/words.txt','r').read().splitlines()

In [ ]:
def get_list_of_words(bag):
  list_of_words = []
  words = get_words()
  for element in bag:
    list_of_words.append(words[int(element,2)])
  return list_of_words

In [ ]:
def split_entropy_bits_finale(entropy_bits_finale):
  binary_seed_phrase = []
  i= 0
  while i < 132:
      binary_seed_phrase.append(entropy_bits_finale[i:i+11])
      i = i + 11
  print("Splitted :")
  print(binary_seed_phrase)
  print("")
  return binary_seed_phrase

In [ ]:
def gen_root_seed(seed_phrase):
  seed_phrase_bytes = seed_phrase.encode("utf-8")
  salt = "mnemonic".encode("utf-8")
  root_seed = hashlib.pbkdf2_hmac(
      "sha512", seed_phrase_bytes, salt, 2048
  )
  print("Root seed :")
  print(root_seed.hex())
  print("")
  return root_seed

In [ ]:
def hmac_sha512(root_seed):
  hashed_root_seed = hmac.new(b"Bitcoin seed", root_seed, digestmod = hashlib.sha512).hexdigest()
  # hashed_root_seed = format(int(hashed_root_seed,16),'0256b')
  print("Output of the master private key in binary :")
  print(hashed_root_seed[:64])
  print("Output of the master chain code in binary :")
  print(hashed_root_seed[64:])
  print(" ")
  return hashed_root_seed[:64], hashed_root_seed[64:]

In [ ]:
def gen_seed_phrase(entropy):
  binary_seed_phrase = split_entropy_bits_finale(add_checksum_to_entropy(entropy))
  seed_phrase = get_list_of_words(binary_seed_phrase)
  seed_phrase = " ".join(seed_phrase)
  print("list of words : ")
  print(seed_phrase)
  print("")
  return seed_phrase

In [125]:
def gen_child_pubkey(parent_key, parent_chain_code, index = None):
  index_binary = ""
  if index:
    index_binary = format(index,'032b')
  else:
    index_binary = format(secrets.randbits(32),'032b')
  elements = parent_key + parent_chain_code + index_binary
  elements = elements.encode("utf-8")
  hashed_root_seed = hmac.new(b"Bitcoin seed", elements, digestmod = hashlib.sha512).hexdigest()
  print(hashed_root_seed)
  print("Output of the children private key in binary :")
  print(hashed_root_seed[:64])
  print("Output of the children chain code in binary :")
  print(hashed_root_seed[64:])
  print(" ")
  return hashed_root_seed[:64], hashed_root_seed[64:]


In [126]:
def gen_children(parent_key, parent_chain_code, deriv, index = None):
  if index:
    index_binary = format(index,'032b')
  else:
    index_binary = format(secrets.randbits(32),'032b')
  index_binary = format(index,'032b')
  elements = parent_key + parent_chain_code + index_binary
  elements = elements.encode("utf-8")
  hashed_root_seed = hmac.new(b"Bitcoin seed", elements, digestmod = hashlib.sha512).hexdigest()
  key = hashed_root_seed[:64]
  code = hashed_root_seed[64:]
  while deriv > 0:
    deriv = deriv-1
    print('plus que ',deriv,'itération avant d atteindre la clé associé a la derrivation voulue')
    return(gen_children(format(int(key,16),'0256b'),format(int(code,16),'0256b'),deriv,index))
  print('l index de la clé est ', index)
  return(key,code)

In [129]:
def main():
  entropy = gen_entropy()
  seed_phrase = gen_seed_phrase(entropy)
  check_entropy()

  master_private_key, master_chain_code = hmac_sha512(gen_root_seed(seed_phrase))
  binary_master_private_key = format(int(master_private_key,16),'0256b')
  binary_master_chain_code = format(int(master_chain_code,16),'0256b')

  child_private_key, child_chain_code = gen_child_pubkey(binary_master_private_key, binary_master_chain_code)
  print('A child private key is', child_private_key, end = '\n\n')

  index = int(input('Please choose an integer between 0 and 4294967295 to generate a children : '))
  child_private_key, child_chain_code = gen_child_pubkey(binary_master_private_key, binary_master_chain_code, index)
  print('The child private key of index', index, ' is', child_private_key, end = '\n\n')

  binary_master_private_key = format(int(master_private_key,16),'0256b')
  binary_master_chain_code = format(int(master_chain_code,16),'0256b')
  index = int(input('Please choose an integer between 0 and 4294967295 to generate a children : '))
  deriv = int(input('Please choose an integer for the level of derivation : '))
  child_private_key, child_chain_code = gen_child_pubkey(binary_master_private_key, binary_master_chain_code, deriv, index)
  print('The child private key of index', index, ' and derivation', deriv, ' is', child_private_key, end = '\n\n')

main()

Random number :
130487633271635353347954927646630823985
 
Random number in binary :
01100010001010110000000100001000000010111000001000101010100011110110101011111010001010010100010010101101110011010100100000110001
 
Output of the hash in binary :
0100100011111111011000111001111011111111101000011010101100000011110110001101011110100001111101111000110100111111000010000000100011100100100111110110000100010010101010011000001001001001100111100010010001111001111010110000101110001000111111001001110001010010
 
Entropy + Checksum :
011000100010101100000001000010000000101110000010001010101000111101101010111110100010100101000100101011011100110101001000001100010100

Splitted :
['01100010001', '01011000000', '01000010000', '00010111000', '00100010101', '01000111101', '10101011111', '01000101001', '01000100101', '01101110011', '01010010000', '01100010100']

list of words : 
giraffe fix dragon blade cargo element program early dwarf hover faith glad

Please type the words of your seed
giraffe fix dragon

TypeError: ignored

In [ ]:
cmd = ['bx', 'ec-to-public', 'b2a0d576b828b537688b561f2cfa8dac3602d54c62bde619ad5331e6c235ee26']
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

o, e = proc.communicate()

print('Output: ' + o.decode('ascii'))
print('Error: '  + e.decode('ascii'))
print('code: ' + str(proc.returncode))